In [33]:
import pandas as pd
import scipy.stats as ss
import numpy as np
import seaborn as sns
from scipy.stats import chi2_contingency

In [34]:
data = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-kuksova/AB-tests/dating_data.csv')
data

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1
...,...,...,...,...
14509,476,631,0,0
14510,455,770,1,0
14511,260,6,1,1
14512,885,812,1,1


In [37]:
data['groupPairs'] = data.user_id_1.map(str) + '/' + data.user_id_2.map(str)
data.drop(columns=['user_id_1', 'user_id_2'])

,group,is_match,groupPairs
0,1,1,79/91
1,1,1,716/353
2,0,0,423/677
3,1,1,658/165
4,0,1,969/155
...,...,...,...
14509,0,0,476/631
14510,1,0,455/770
14511,1,1,260/6
14512,1,1,885/812


In [44]:
data.query('group == 1').nunique()[4] # количество наблюдений в группе 1

9722

In [51]:
data.query('group == 0').nunique()[4] # количество наблдюдений в группе 0

4792

Проверим математически и постараемся оценить значимость обновления:

In [72]:
data.query('group == 1 and is_match == 1').nunique()[4] / data.query('group == 1').nunique()[4] 
# доля удачных исходов в группе с новой системой

0.40207776177741206

In [71]:
data.query('group == 0 and is_match == 1').nunique()[4] / data.query('group == 0').nunique()[4] 
# доля удачных исходов в группе со старой системой

0.19490818030050083

Математически соотношение успешных исходов в группе с новым алгоритмом для поиска анкет превалирует.

________________________________________________________________________________________________________________________________

Проверим статистически:

Исходя из того, что у нас есть две группы пользователей (группа 0 и группа 1), и для каждой группы есть бинарная метрика (1 - удачный match, 0 - неудачный match), можно использовать критерий хи-квадрат (Chi-squared test) для сравнения доли удачных действий между группами. Этот критерий подходит для анализа бинарных данных и определения, есть ли статистически значимые различия в распределении бинарных исходов между группами.
    
Критерий хи-квадрат проверяет следующие гипотезы:

**Гипотеза H0: Доли удачных мэтчей в группе 0 и группе 1 равны.**

**Гипотеза H1: Доли мэтчей действий в группах 0 и 1 различаются.**

Проверяем!

Процедура анализа включает в себя создание таблицы сопряженности (кросс-таблицы), где строки представляют группы (0 и 1), а столбцы представляют значения бинарной метрики (1 и 0). Затем вычисляется статистика хи-квадрат и соответствующее p-значение.

In [64]:
dat_cross = pd.crosstab(data.group, data.is_match) # строим таблицу сопряженности
dat_cross

is_match,0,1
group,,
0,3858,934
1,5813,3909


In [50]:
# применяем критерий хи-квадрат
chi2, p, dof, expected = chi2_contingency(dat_cross)
print("Chi-squared statistic:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies table:")
print(expected)

Chi-squared statistic: 618.6889141576198
p-value: 1.4418299163662586e-136
Degrees of freedom: 1
Expected frequencies table:
[[3193.01584677 1598.98415323]
 [6477.98415323 3244.01584677]]


    p-value: 1.4418299163662586e-136

#### Заключение:

P-value, полученное в результате анализа, меньше выбранного уровня значимости (α = 0.05), следовательно мы можем отвергнуть нулевую гипотезу и заключить, что доли удачных мэтчей в группах различаются статистически значимо.

**Ответ на главный вопрос, стоит ли включать новую систему поиска анкет на всех пользователей - можно попробовать :)